In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read data

In [2]:
data_path = './preprocessed data/'

In [3]:
df_train = {}
impute =  os.listdir(data_path)
for method in impute:
    goto = data_path + str(method)
    method = method.replace(".csv","")

    if method == 'drop_all_nan':
        df_raw = pd.read_csv(goto, index_col=0).reset_index().round(3).iloc[:,1:]
        df_raw = df_raw[['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','0','1','2','3','4','5','6']]

    elif method == 'ctgan':
        df_train[method] = pd.read_csv(goto, index_col=0)
        df_train[method] = df_train[method][['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','0','1','2','3','4','5','6']]
    
    else:
        df_train[method] = pd.read_csv(goto, index_col=0).reset_index().round(3).iloc[:,3:]
        df_train[method] = df_train[method][['7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','0','1','2','3','4','5','6']]


In [4]:
print(df_train.keys())

dict_keys(['ctgan', 'four_avg_drop_nan', 'knn', 'mice', 'three_next_avg_drop_nan', 'three_pre_avg_drop_nan', 'two_avg_drop_nan', 'two_weighted_drop_nan'])


# Split x, y

In [5]:
X = {}
y = {}

X["raw"] = df_raw.iloc[:,:15].values
y["raw"] = df_raw.iloc[:,15:].values
for method in df_train.keys():
    X[method] = df_train[method].iloc[:,:15].values
    y[method] = df_train[method].iloc[:,15:].values

# RNN model

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, SimpleRNN, LSTM, GRU, Dropout, Dense
from sklearn.model_selection import KFold

In [7]:
def create_rnn_model(timesteps, features, verbose):
    model = Sequential([
        Bidirectional(SimpleRNN(50, activation='relu', input_shape=(timesteps, features), return_sequences=True)),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        GRU(50, activation='relu', return_sequences=False),
        Dense(9)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    if verbose:
        print(model.summary())
    return model

In [8]:
def model_train(X, y, loss, train_size):
    for method in df_train.keys():

        # 把一半的完整資料加進去訓練集
        X_impute, y_impute = X[method], y[method]
        X_train, y_train = np.concatenate((X_impute, X["raw"][:71]), axis=0), np.concatenate((y_impute, y["raw"][:71]), axis=0)
        X_test, y_test = X["raw"][71:], y["raw"][71:]

        # Reshape input X into a 3D tensor with shape (num_samples, timesteps, features_per_timestep)
        X_train_3d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test_3d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

        # Create and train the RNN model
        model = create_rnn_model(X_train_3d.shape[1], X_train_3d.shape[2],verbose=False)
        model.fit(X_train_3d, y_train, epochs=50, batch_size=16)

        # Evaluate the model on the test set
        test_loss = model.evaluate(X_test_3d, y_test)
        loss[method] = test_loss
        train_size[method] = len(X_train)


In [9]:
def model_train_raw(X, y, loss, train_size):

    # 把一半的完整資料加進去訓練集
    X_train, y_train = X["raw"][:71], y["raw"][:71]
    X_test, y_test = X["raw"][71:], y["raw"][71:]

    # Reshape input X into a 3D tensor with shape (num_samples, timesteps, features_per_timestep)
    X_train_3d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test_3d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Create and train the RNN model
    model = create_rnn_model(X_train_3d.shape[1], X_train_3d.shape[2],verbose=False)
    model.fit(X_train_3d, y_train, epochs=50, batch_size=16)

    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test_3d, y_test)
    loss["raw"] = test_loss
    train_size["raw"] = len(X_train)


In [10]:
loss = {}
train_size = {}
model_train(X, y, loss, train_size)



Epoch 1/50

14/14 [==============================] - 7s 19ms/step - loss: 17237.4453
Epoch 2/50
14/14 [==============================] - 0s 16ms/step - loss: 17038.5098
Epoch 3/50
14/14 [==============================] - 0s 18ms/step - loss: 15745.6230
Epoch 4/50
14/14 [==============================] - 0s 19ms/step - loss: 5269.7422
Epoch 5/50
14/14 [==============================] - 0s 19ms/step - loss: 1652.9474
Epoch 6/50
14/14 [==============================] - 0s 20ms/step - loss: 970.0635
Epoch 7/50
14/14 [==============================] - 0s 19ms/step - loss: 838.5027
Epoch 8/50
14/14 [==============================] - 0s 19ms/step - loss: 786.0029
Epoch 9/50
14/14 [==============================] - 0s 19ms/step - loss: 791.5732
Epoch 10/50
14/14 [==============================] - 0s 19ms/step - loss: 778.9796
Epoch 11/50
14/14 [==============================] - 0s 24ms/step - loss: 788.0248
Epoch 12/50
14/14 [==============================] - 0s 24ms/step - loss: 778.7924
Ep

In [11]:
model_train_raw(X, y, loss, train_size)

Epoch 1/50
5/5 [==============================] - 16s 23ms/step - loss: 14602.7900
Epoch 2/50
5/5 [==============================] - 0s 22ms/step - loss: 14549.8105
Epoch 3/50
5/5 [==============================] - 0s 22ms/step - loss: 14522.7305
Epoch 4/50
5/5 [==============================] - 0s 22ms/step - loss: 14494.2988
Epoch 5/50
5/5 [==============================] - 0s 24ms/step - loss: 14460.7539
Epoch 6/50
5/5 [==============================] - 0s 24ms/step - loss: 14415.5791
Epoch 7/50
5/5 [==============================] - 0s 24ms/step - loss: 14338.6865
Epoch 8/50
5/5 [==============================] - 0s 24ms/step - loss: 14210.2529
Epoch 9/50
5/5 [==============================] - 0s 24ms/step - loss: 13955.5830
Epoch 10/50
5/5 [==============================] - 0s 25ms/step - loss: 13369.5166
Epoch 11/50
5/5 [==============================] - 0s 25ms/step - loss: 11882.8867
Epoch 12/50
5/5 [==============================] - 0s 30ms/step - loss: 7160.0903
Epoch 13/50
5

In [12]:
loss.keys()

dict_keys(['ctgan', 'four_avg_drop_nan', 'knn', 'mice', 'three_next_avg_drop_nan', 'three_pre_avg_drop_nan', 'two_avg_drop_nan', 'two_weighted_drop_nan', 'raw'])

In [13]:
loss

{'ctgan': 437.6817932128906,
 'four_avg_drop_nan': 159.98655700683594,
 'knn': 360.2275695800781,
 'mice': 356.1040954589844,
 'three_next_avg_drop_nan': 369.0331115722656,
 'three_pre_avg_drop_nan': 310.2837219238281,
 'two_avg_drop_nan': 153.39662170410156,
 'two_weighted_drop_nan': 328.3555908203125,
 'raw': 353.9844970703125}

In [14]:
train_size

{'ctgan': 213,
 'four_avg_drop_nan': 1357,
 'knn': 1530,
 'mice': 1530,
 'three_next_avg_drop_nan': 1333,
 'three_pre_avg_drop_nan': 1356,
 'two_avg_drop_nan': 1330,
 'two_weighted_drop_nan': 597,
 'raw': 71}

In [15]:
# num_rows = 6
# num_cols = 5
# fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))
# axes = axes.flatten() # for easier indexing

# predictions = best_model.predict(best_X_test)

# # Loop through the predictions and actual values
# for i in range(len(X_test)):
#     ax = axes[i]
#     ax.plot(predictions[i].tolist(), label='Predictions')

#     # Plot actual values
#     ax.plot(best_y_test[i].tolist(), label='Actual Values')

#     # Customize the subplot
#     ax.set_title(f'Row {best_test_index[i]}')
#     ax.set_xlabel('Timestep')
#     ax.set_ylabel('Value')
#     # ax.legend()

# # Use tight layout to ensure proper spacing
# plt.tight_layout()

# # Show or save the plot
# plt.show()

# 計算 bp>125 的個數

In [16]:
# def nighttime_hpt_count(y, lst):
#     for i in range(len(y)):
#         lst.append(sum(1 if k>125 else 0 for k in y[i]))

In [17]:
# pred_count = []
# true_count = []
# nighttime_hpt_count(predictions, pred_count)
# nighttime_hpt_count(best_y_test, true_count)

In [18]:
# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(pred_count, true_count)
# print(f'Mean Squared Error: {mse}')